Import packages

In [1]:
import requests
from bs4 import BeautifulSoup

import re, ast, json
import requests
import subprocess
import json
from bs4 import BeautifulSoup

import json
import re
from collections import defaultdict

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN

Define sources from web for scraping & fetching data

In [2]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

SHOP_SITES_LIST = {
    "neptun": [
        "https://www.neptun.mk/mobilni_telefoni.nspx?items=100&page=1",
        "https://www.neptun.mk/mobilni_telefoni.nspx?items=100&page=2",
        "https://www.neptun.mk/mobilni_telefoni.nspx?items=100&page=3",
    ],
    "telekom": [
        "https://www.telekom.mk/services/DeviceListService.asmx/GetDevices"
    ],
    "anhoch": [
        "https://www.anhoch.com/products?"
        "query=&"
        "categories%5B0%5D=mobilni-telefoni&"
        "tag=&"
        "fromPrice=0&"
        "toPrice=324980&"
        "inStockOnly=2&"
        "sort=latest&"
        "perPage=50&page=1",
        "https://www.anhoch.com/products?"
        "query=&"
        "categories%5B0%5D=mobilni-telefoni&"
        "tag=&"
        "fromPrice=0&"
        "toPrice=324980&"
        "inStockOnly=2&"
        "sort=latest&"
        "perPage=50&page=2",
        "https://www.anhoch.com/products?"
        "query=&"
        "categories%5B0%5D=mobilni-telefoni&"
        "tag=&"
        "fromPrice=0&"
        "toPrice=324980&"
        "inStockOnly=2&"
        "sort=latest&"
        "perPage=50&page=3",
        "https://www.anhoch.com/products?"
        "query=&"
        "categories%5B0%5D=mobilni-telefoni&"
        "tag=&"
        "fromPrice=0&"
        "toPrice=324980&"
        "inStockOnly=2&"
        "sort=latest&"
        "perPage=50&page=4",
        "https://www.anhoch.com/products?"
        "query=&"
        "categories%5B0%5D=mobilni-telefoni&"
        "tag=&"
        "fromPrice=0&"
        "toPrice=324980&"
        "inStockOnly=2&"
        "sort=latest&"
        "perPage=50&page=5",
    ]
}

Scrape from Neptun

In [3]:
def scrape_neptun():
    urls = SHOP_SITES_LIST.get("neptun", [])
    if isinstance(urls, str):
        urls = [urls]

    all_items = []
    for page in urls:
        resp = requests.get(page, headers=HEADERS)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")

        script = soup.find("script", string=re.compile(r"shopCategoryModel"))
        if not script:
            continue

        m = re.search(
            r"var\s+shopCategoryModel\s*=\s*(?P<quote>['\"])(?P<content>.+?)(?P=quote);",
            script.string,
            re.DOTALL
        )
        if not m:
            continue

        # turn the JS string literal into JSON
        json_str = ast.literal_eval(m.group("quote") + m.group("content") + m.group("quote"))
        data     = json.loads(json_str)

        for p in data.get("Products", []):
            all_items.append({
                "origin":        "neptun",
                "product_origin_name":  p.get("Title"),
                "product_origin_price": p.get("ActualPrice") or p.get("RegularPrice"),
                "product_origin_url":   "https://www.neptun.mk/categories/mobilni_telefoni/" + p.get("Url", "").lstrip("/")
            })

    return all_items

Scrape from Anhoch

In [4]:
def scrape_anhoch():
    urls = SHOP_SITES_LIST["anhoch"]
    all_items = []
    for url in urls:
        cmd = [
            "curl", "-s", url,
            "-H", "Accept: */*",
            "-H", "Accept-Language: en-US,en;q=0.9,mk;q=0.8",
            "-H", "X-Requested-With: XMLHttpRequest",
        ]
        proc = subprocess.run(cmd, capture_output=True, text=True)
        proc.check_returncode()

        payload  = json.loads(proc.stdout)
        products = payload.get("products", {}).get("data", [])

        for p in products:
            all_items.append({
                "origin":               "anhoch",
                "product_origin_name":  p.get("name"),
                "product_origin_url":   p.get("slug"),
                "product_origin_price": p.get("selling_price", {}).get("amount")
            })

    return all_items

Scrape from Telekom MK

In [5]:
def scrape_telekom():
    url = SHOP_SITES_LIST["telekom"][0]

    data_raw = json.dumps({
        "model": {
            "DeviceType": {
                "Id": 3,
                "Description": "Мобилни уреди",
                "IsCollect": False
            },
            "OnlinePromotion": {
                "Description": "Промоции",
                "DescriptionEn": "Promotions",
                "DescriptionAl": "All",
                "NumberOfDevices": 0,
                "ShowInFilter": True,
                "ShowInBusiness": True,
                "UserTypeId": 0,
                "Id": 0,
                "Value": "Сите",
                "ValueEn": "All",
                "ValueAl": "All"
            },
            "OnlineAvailability": {
                "Description": "Онлајн достапност",
                "DescriptionEn": "Online availability",
                "DescriptionAl": "Në dispozicion onlajn",
                "Id": 0,
                "Value": "Сите",
                "ValueEn": "All",
                "ValueAl": "Të gjitha"
            },
            "SortType": {
                "Id": 1,
                "Value": "Хронолошки",
                "ValueEn": "Hronological",
                "ValueAl": "Kronologjike"
            },
            "SortTypeId": 1,
            "PagingType": {
                "Id": 0,
                "Value": "Сите",
                "ValueEn": "All",
                "ValueAl": "Të gjitha"
            },
            "PaymentType": {
                "Id": 1,
                "Description": "Еднократно",
                "IsCollect": False
            },
            "PaymentTypeId": 1,
            "Manufacturers": [],
            "Memories": [],
            "PriceRanges": [],
            "NetworkType": [],
            "ProductType": 3,
            "TariffGroupId": "",
            "TariffId": "",
            "TariffName": "",
            "PrepaidOption": None,
            "OnlinePromotionName": "",
            "SelectedColor": "",
            "SelectedPackageType": "postpaid",
            "DetailsUrl": "mobile-device-details.nspx",
            "IsBussiness": False,
            "FamilyBudgetPayment": {
                "Id": 2,
                "Description": "Буџет",
                "IsCollect": False
            },
            "FamilyBudgetPaymentId": 2,
            "ItemsPerPage": 0,
            "ClickAndCollect": {
                "Id": 1,
                "Description": "Достава на адреса",
                "IsCollect": True
            }
        }
    }, ensure_ascii=False)

    cmd = [
        "curl", "-s", url,
        "-H", "Accept: application/json, text/plain, */*",
        "-H", "Content-Type: application/json;charset=UTF-8",
        "-H", "Origin: https://www.telekom.mk",
        "-H", "Referer: https://www.telekom.mk/mobilni-uredi.nspx?priceType=1&familijarenBudzet=2&deviceType=3&collect=true",
        "--data-raw", data_raw
    ]

    proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    proc.check_returncode()

    raw = proc.stdout.decode("utf-8", errors="ignore")
    payload = json.loads(raw)

    devices = payload.get("d", {}).get("Devices", [])
    result = []
    for d in devices:
        result.append({
            "origin":               "telekom",
            "product_origin_name":  d.get("Name"),
            "product_origin_price": d.get("Price", {}).get("PriceNormal"),
            "product_origin_url":   d.get("DetailsUrl")
        })

    return result

In one array append all scraped data from all of the different sources

In [6]:
all_items = []
all_items.extend(scrape_neptun())
all_items.extend(scrape_anhoch())
all_items.extend(scrape_telekom())

In [7]:
#all_items

Group by manufacturer

In [8]:
grouped = {}
for item in all_items:
    name = item.get("product_origin_name", "")
    if not name:
        continue
    key = name.split()[0].lower()  # first word, case-insensitive
    grouped.setdefault(key, []).append(item)

grouped_list = [
    {
        "first_word": first_word,
        "products": products
    }
    for first_word, products in grouped.items()
]

Consolidate groupings by manufacturer (ex. make apple and iphone one group)

In [9]:
consolidated = {}

for first_word, products in grouped.items():
    # drop unwanted groups entirely
    if first_word in ("smartphone", "hmd", "tcl", "alcatel"):
        continue

    # merge Apple and iPhone into one bucket
    if first_word in ("apple", "iphone"):
        consolidated.setdefault("apple_iphone", []).extend(products)

    # merge Xiaomi and Redmi into another bucket
    elif first_word in ("xiaomi", "redmi"):
        consolidated.setdefault("xiaomi_redmi", []).extend(products)

    # keep all other groups as-is
    else:
        consolidated.setdefault(first_word, []).extend(products)

Group by colors and intenral storage

In [14]:
data = consolidated

COLORS = {
    'black','white','blue','red','green','gold','silver','purple','lavender','pink',
    'mint','yellow','gray','grey','navy','pearl','teal','cyan','olive','rose',
    'midnight','graphite','titanium','ultramarine','starlight','onyx'
}

SPEC_PATTERNS = [
    # match "3+64gb" or "128gb" or "1tb"
    r'\b\d+(?:\+\d+)?(?:gb|tb)\b',
    # match screen-inches like 6.5" or 6.7”
    r'\b\d+(\.\d+)?["”]\b',
    # match cameras like 108MP
    r'\b\d+mp\b',
    # match battery specs like 5000mAh
    r'\b\d+mah\b',
]

def clean_name(raw, brand_tokens):
    s = raw.lower()
    # 1) remove parentheticals
    s = re.sub(r'\([^)]*\)', '', s)
    # 2) drop anything after a slash
    s = s.split('/', 1)[0]
    # 3) strip all spec patterns (case-insensitive)
    for pat in SPEC_PATTERNS:
        s = re.sub(pat, '', s, flags=re.IGNORECASE)
    # 4) drop brand tokens (e.g. "samsung", "galaxy", "xiaomi", "redmi", "mi", "iphone", "apple")
    for tok in brand_tokens:
        s = re.sub(rf'\b{re.escape(tok.lower())}\b', '', s)
    # 5) drop color words
    for col in COLORS:
        s = re.sub(rf'\b{re.escape(col)}\b', '', s)
    # 6) keep only letters, digits, spaces, "+" and "g" (so "5g" survives)
    s = re.sub(r'[^a-z0-9 +g]', ' ', s)
    # 7) collapse whitespace
    s = re.sub(r'\s+', ' ', s).strip()
    # 8) take only the first TWO tokens (e.g. "s24fe 5g", "note 13", "10t")
    tokens = s.split()
    return ' '.join(tokens[:2])

# Cluster per manufacturer
grouped = {}

for mfr, products in data.items():
    # build a list of tokens to drop for this brand
    brand_tokens = mfr.replace('_', ' ').split()
    # special case: Xiaomi/Redmi also sometimes called "Mi"
    if mfr == 'xiaomi_redmi':
        brand_tokens.append('mi')

    # clean every product name
    cleaned = [clean_name(p['product_origin_name'], brand_tokens)
               for p in products]

    # vectorize on character n-grams
    vec = TfidfVectorizer(analyzer='char_wb', ngram_range=(3,5))
    X = vec.fit_transform(cleaned)

    # cluster via DBSCAN (cosine metric)
    labels = DBSCAN(metric='cosine', eps=0.1, min_samples=1).fit_predict(X)

    # collect clusters
    clusters = defaultdict(list)
    for lbl, prod in zip(labels, products):
        clusters[lbl].append(prod)

    # pick a canonical cleaned name per cluster
    canon = {}
    for lbl, members in clusters.items():
        # find one of the cleaned names for this label
        names = [cleaned[i] for i,p in enumerate(products) if labels[i]==lbl]
        canon[lbl] = min(filter(None, names), key=len)

    # rebuild final grouped dict
    grouped[mfr] = { canon[l]: cluster for l, cluster in clusters.items() }

In [11]:
def group_by_origin(data):
    grouped = {}
    for brand, models in data.items():
        grouped[brand] = {}
        for model_name, items in models.items():
            by_origin = defaultdict(list)
            for item in items:
                src = item.get('origin', 'other')
                by_origin[src].append(item)
            grouped[brand][model_name] = dict(by_origin)
    return grouped

In [16]:
group_by_origin(grouped)

{'nokia': {'105 1': {'neptun': [{'origin': 'neptun',
     'product_origin_name': 'Nokia 105, 1.8", 1000mAh, Black',
     'product_origin_price': 1599.0,
     'product_origin_url': 'https://www.neptun.mk/categories/mobilni_telefoni/Nokia-105-Black--1GF019CPA2L03-'},
    {'origin': 'neptun',
     'product_origin_name': 'Nokia 105, 1.8", 1000mAh, Black',
     'product_origin_price': 1599.0,
     'product_origin_url': 'https://www.neptun.mk/categories/mobilni_telefoni/Nokia-105-Black--1GF019CPA2L03-'}]},
  '105 ds': {'neptun': [{'origin': 'neptun',
     'product_origin_name': 'NOKIA 105 DS 2023 Cyan (1GF019CPG6L03)',
     'product_origin_price': 1599.0,
     'product_origin_url': 'https://www.neptun.mk/categories/mobilni_telefoni/NOKIA-105-DS-2023-Cyan--1GF019CPG6L03-'},
    {'origin': 'neptun',
     'product_origin_name': 'NOKIA 105 DS 2023 Cyan (1GF019CPG6L03)',
     'product_origin_price': 1599.0,
     'product_origin_url': 'https://www.neptun.mk/categories/mobilni_telefoni/NOKIA-105-DS